In [47]:
import pandas as pd
import tweepy as tw
import webbrowser
import time

from config import api_key
from config import api_key_secret
from config import bearer_token
from config import access_token
from config import access_token_secret

In [48]:
auth = tw.OAuth1UserHandler(
   api_key, api_key_secret, access_token, access_token_secret
)

api = tw.API(auth)

print(api.verify_credentials().screen_name)


momsthatcode90s


In [49]:
client = tw.Client(bearer_token=bearer_token)

In [50]:
client = tw.Client(
    consumer_key=api_key, consumer_secret=api_key_secret,
    access_token=access_token, access_token_secret=access_token_secret
)

## user info

In [51]:
user = 'elonmusk'

In [ ]:
# Code Source
#https://towardsdatascience.com/how-to-use-twitter-premium-search-apis-for-mining-tweets-2705bbaddca


## Full Archive Method

In [52]:
# Code reproduced from Twitter developer API documentation [4]
def determine_tweet_type(tweet):
    # Check for reply indicator first
    if tweet["in_reply_to_status_id"] is not None:
        tweet_type = "Reply Tweet"
    # Check boolean quote status field but make sure it's not a Retweet (of a Quote Tweet) 
    elif tweet["is_quote_status"] is True and not tweet["text"].startswith("RT"):
        tweet_type = "Quote Tweet"
    # Check both indicators of a Retweet
    elif tweet["text"].startswith("RT") and tweet.get("retweeted_status") is not None:
        tweet_type = "Retweet"
    else:
        tweet_type = "Original Tweet"
    return tweet_type 

In [53]:
def parse_tweets(status):
    for tweet in status:
        if determine_tweet_type(tweet._json) == 'Retweet':
            if 'extended_tweet' in tweet._json['retweeted_status']:
                full_text = tweet._json['retweeted_status']['extended_tweet']['full_text']
            else:
                full_text = tweet._json['retweeted_status']['text']
               

        elif determine_tweet_type(tweet._json) == 'Quote Tweet':
            if 'extended_tweet' in tweet._json['quoted_status']:
                full_text = tweet._json['quoted_status']['extended_tweet']['full_text']
            else:
                full_text = tweet._json['quoted_status']['text']
                
        else:
            if 'extended_tweet' in tweet._json:
                full_text = tweet._json['extended_tweet']['full_text']
            else:
                full_text = tweet._json['text']

        mydict = { "tweet_id": tweet._json["id_str"],
                       "date":tweet._json["created_at"],
                       "full_text": full_text,
                       "tweet_type": determine_tweet_type(tweet._json),
                        "reply_count": tweet._json["reply_count"], #Number of times Tweet has been replied to
                       "quote_count": tweet._json["quote_count"], # Number of times Tweet has been quoted
                       "likes_count": tweet._json["favorite_count"], #Number of times Tweet has been liked 
                       "retweet_counts": tweet._json["retweet_count"], #Number of times this Tweet has been retweeted
                       "hyperlink": "https://twitter.com/twitter/status/" + tweet._json["id_str"]
              }
        parsedTweets.append(mydict) # Add Tweet to parsedTweets list
            

In [54]:
parsedTweets = []
#https://twittercommunity.com/t/correct-syntax-for-an-exact-phrase-match-and-keyword-query/124617
status = tw.Cursor(api.search_full_archive,label='SearchElon', 
                          query= "\"@elonmusk\" \"dogecoin\"", # To match a phrase, you’d use quotes but you need to escape them since it’s a json value,
                          fromDate='201401010000',
                          toDate='202207070000').items(1000)
parse_tweets(status)

tweetdf = pd.DataFrame(parsedTweets)


In [57]:
status

In [55]:
tweetdf

,tweet_id,date,full_text,tweet_type,reply_count,quote_count,likes_count,retweet_counts,hyperlink
0,1544833050814685184,Wed Jul 06 23:57:55 +0000 2022,Good news for #dogecoin and #Crypto. #ElonMusk...,Original Tweet,0,0,0,0,https://twitter.com/twitter/status/15448330508...
1,1544833049187258369,Wed Jul 06 23:57:55 +0000 2022,CNN is reporting that @elonmusk's @boringcompa...,Retweet,0,0,0,0,https://twitter.com/twitter/status/15448330491...
2,1544833034762993665,Wed Jul 06 23:57:51 +0000 2022,@elonmusk Dogecoin Earther,Retweet,0,0,0,0,https://twitter.com/twitter/status/15448330347...
3,1544832945642405889,Wed Jul 06 23:57:30 +0000 2022,@UpdateRail @elonmusk @inevitable360 @ArroyoXI...,Reply Tweet,0,0,12,5,https://twitter.com/twitter/status/15448329456...
4,1544832835294756864,Wed Jul 06 23:57:04 +0000 2022,@mishaboar @boringcompany Supporting Doge wher...,Quote Tweet,1,0,3,0,https://twitter.com/twitter/status/15448328352...
...,...,...,...,...,...,...,...,...,...
95,1544823349863280640,Wed Jul 06 23:19:22 +0000 2022,@Dogemarine @mishaboar @markyKG @elonmusk @bor...,Retweet,0,0,0,0,https://twitter.com/twitter/status/15448233498...
96,1544823090244460545,Wed Jul 06 23:18:20 +0000 2022,Norwegian Tunnel Elon 😀👊 cant use #dogecoin bu...,Retweet,0,0,0,0,https://twitter.com/twitter/status/15448230902...
97,1544822929375907840,Wed Jul 06 23:17:42 +0000 2022,CNN is reporting that @elonmusk's @boringcompa...,Retweet,0,0,0,0,https://twitter.com/twitter/status/15448229293...
98,1544822925089546240,Wed Jul 06 23:17:41 +0000 2022,Labrado Inu got some special snack to celebrat...,Original Tweet,2,0,7,2,https://twitter.com/twitter/status/15448229250...
